In [2]:
# Import the class from the Python file (module)
import pandas as pd
import matplotlib.pyplot as plt
import os
# from dotenv import load_dotenv
# from pathlib import Path
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from BinanceClient import BinanceClient
import numpy as np
from typing import Final
import joblib
from BatchFeatures import BatchFeatures
from datetime import datetime, timedelta
%matplotlib widget

## Load pair df

In [3]:
import os
from datetime import datetime, timedelta, timezone

def interval_slug(s: str) -> str:
    return s.strip().replace(" ", "").replace("/", "").lower()

def make_db_name(pair: str, interval: str, weeks: int) -> str:
    return f"{pair}_{interval_slug(interval)}_{weeks}weeks.db"

def load_or_fetch_pair_df(pair: str, interval: str, weeks: int) -> tuple[str, "pd.DataFrame"]:
    db_name = make_db_name(pair, interval, weeks)
    db_path = db_name

    print(f"[{pair}] DB: {db_path}")

    binance_client = BinanceClient(db_path)
    binance_client.set_interval(interval)

    df = None

    if os.path.exists(db_path):
        df = binance_client.fetch_data_from_db(pair)
        if df is not None and not df.empty:
            print(f"[{pair}] Loaded {len(df):,} rows from DB.")
        else:
            df = None

    if df is None:
        print(f"[{pair}] No usable DB data found -> fetching from Binance...")

        api_secret = os.getenv("BINANCE_SECRET_KEY")
        api_key = os.getenv("BINANCE_API_KEY")
        binance_client.make(api_key, api_secret)

        server_time = binance_client.get_server_time()
        end_dt = datetime.fromtimestamp(server_time["serverTime"] / 1000, tz=timezone.utc)
        start_dt = end_dt - timedelta(weeks=weeks)

        start_ms = int(start_dt.timestamp() * 1000)
        end_ms = int(end_dt.timestamp() * 1000)

        data = binance_client.fetch_data(pair, start_ms, end_ms)
        if data is None or data.empty:
            raise RuntimeError(f"[{pair}] No data returned from Binance for the requested range.")

        binance_client.store_data_to_db(pair, data)

        df = binance_client.fetch_data_from_db(pair)
        if df is None or df.empty:
            raise RuntimeError(f"[{pair}] Data fetched/stored but DB load returned empty.")

        print(f"[{pair}] Fetched + stored + loaded {len(df):,} rows.")

    df = df.sort_index()
    return db_path, df


## Load COINS, then align timestamps

In [9]:
import pandas as pd

interval = "5m"
weeks = 52
# pairs = ["BTCUSDT", "ETHUSDT"]

symbol = [
    "BTCUSDT","ETHUSDT","BNBUSDT","SOLUSDT","XRPUSDT",
    "ADAUSDT","DOGEUSDT","TRXUSDT","LTCUSDT","LINKUSDT"
]
paths = {}
dfs = {}

for sym in symbol:
    db_path, df = load_or_fetch_pair_df(sym, interval, weeks)
    paths[sym] = db_path
    dfs[sym] = df


[BTCUSDT] DB: BTCUSDT_5m_52weeks.db
[BTCUSDT] Loaded 104,832 rows from DB.
[ETHUSDT] DB: ETHUSDT_5m_52weeks.db
[ETHUSDT] Loaded 104,832 rows from DB.
[BNBUSDT] DB: BNBUSDT_5m_52weeks.db
[BNBUSDT] Loaded 104,832 rows from DB.
[SOLUSDT] DB: SOLUSDT_5m_52weeks.db
[SOLUSDT] Loaded 104,832 rows from DB.
[XRPUSDT] DB: XRPUSDT_5m_52weeks.db
[XRPUSDT] Loaded 104,832 rows from DB.
[ADAUSDT] DB: ADAUSDT_5m_52weeks.db
[ADAUSDT] Loaded 104,832 rows from DB.
[DOGEUSDT] DB: DOGEUSDT_5m_52weeks.db
[DOGEUSDT] Loaded 104,832 rows from DB.
[TRXUSDT] DB: TRXUSDT_5m_52weeks.db
[TRXUSDT] Loaded 104,832 rows from DB.
[LTCUSDT] DB: LTCUSDT_5m_52weeks.db
[LTCUSDT] Loaded 104,832 rows from DB.
[LINKUSDT] DB: LINKUSDT_5m_52weeks.db
[LINKUSDT] Loaded 104,832 rows from DB.


In [10]:
import pandas as pd
import numpy as np

def detect_volume_events(
    df: pd.DataFrame,
    symbol: str,
    vol_win: int = 144,          # 12 hours on 5m
    impulse_k: int = 12,         # 60 min impulse
    rvol_thresh: float = 6.0,    # strict
    impulse_thresh: float = 0.04,# +4% over impulse_k
    lookahead: int = 24,         # 2 hours forward path
    cooldown: int = 12,          # avoid logging same burst repeatedly (60 min)
):
    """
    Logs candidate 'flow shock' events:
      - RVOL spike relative to rolling median
      - Positive impulse over last impulse_k bars
    Then measures forward path stats over lookahead bars.
    """
    d = df.copy().sort_index()
    d = d[["open","high","low","close","volume"]].dropna()

    vol_med = d["volume"].rolling(vol_win).median()
    rvol = d["volume"] / vol_med
    impulse = d["close"] / d["close"].shift(impulse_k) - 1.0

    out = []
    i = 0
    n = len(d)

    while i < n - lookahead:
        if (rvol.iloc[i] >= rvol_thresh) and (impulse.iloc[i] >= impulse_thresh):
            px0 = float(d["close"].iloc[i])
            ts0 = d.index[i]

            future = d["close"].iloc[i+1:i+1+lookahead]
            fmax = float(future.max())
            fmin = float(future.min())
            max_fwd_return = fmax / px0 - 1.0
            max_drawdown = fmin / px0 - 1.0

            # retrace from the peak within the lookahead window
            # find peak time then worst after that peak
            peak_idx = future.values.argmax()
            peak_px = float(future.iloc[peak_idx])
            after_peak = future.iloc[peak_idx:]  # includes peak bar
            trough_after_peak = float(after_peak.min())
            max_retrace = trough_after_peak / peak_px - 1.0  # negative means retrace

            # time to max retrace (bars after event)
            trough_idx = after_peak.values.argmin()
            time_to_max_retrace_bars = int(peak_idx + trough_idx + 1)

            out.append({
                "symbol": symbol,
                "event_ts": ts0,
                "close_event": px0,
                "rvol": float(rvol.iloc[i]),
                "impulse": float(impulse.iloc[i]),
                "max_fwd_return": max_fwd_return,
                "max_drawdown": max_drawdown,
                "max_retrace": max_retrace,
                "time_to_max_retrace_bars": time_to_max_retrace_bars,
            })

            i += cooldown  # skip ahead so we don't log every bar of the same burst
        else:
            i += 1

    return pd.DataFrame(out)


In [15]:
# Convert list of DataFrames into a single DataFrame
events = pd.concat(events, ignore_index=True)

type(events), events.head()


(pandas.core.frame.DataFrame,
     symbol            event_ts  close_event       rvol   impulse  \
 0  BTCUSDT 2025-03-02 16:15:00     89370.23  35.169505  0.049267   
 1  BTCUSDT 2025-03-02 17:20:00     92623.03  24.089593  0.040878   
 2  BTCUSDT 2025-04-07 14:15:00     80243.10  12.829945  0.040335   
 3  BTCUSDT 2025-04-09 17:20:00     80744.00  27.602926  0.040441   
 4  BTCUSDT 2025-10-10 22:15:00    113213.78   6.726021  0.088853   
 
    max_fwd_return  max_drawdown  max_retrace  time_to_max_retrace_bars  
 0        0.058518     -0.004590    -0.012570                        23  
 1        0.021344     -0.001134    -0.022009                        18  
 2       -0.010208     -0.030371    -0.020371                        11  
 3        0.026891      0.000949    -0.013591                        19  
 4        0.012397     -0.012771    -0.024860                        24  )

In [16]:
events["symbol"].value_counts()


symbol
DOGEUSDT    37
LINKUSDT    31
LTCUSDT     28
XRPUSDT     27
ADAUSDT     25
SOLUSDT     22
ETHUSDT     17
BTCUSDT      5
BNBUSDT      4
TRXUSDT      2
Name: count, dtype: int64

In [17]:
events.groupby("symbol")[[
    "rvol",
    "impulse",
    "max_fwd_return",
    "max_drawdown",
    "max_retrace",
    "time_to_max_retrace_bars"
]].median()


,rvol,impulse,max_fwd_return,max_drawdown,max_retrace,time_to_max_retrace_bars
symbol,,,,,,
ADAUSDT,11.166297,0.046968,0.020119,-0.011202,-0.019602,21.0
BNBUSDT,9.286704,0.047665,0.016355,-0.008791,-0.021239,17.5
BTCUSDT,24.089593,0.040878,0.021344,-0.004590,-0.020371,19.0
DOGEUSDT,12.303964,0.045536,0.010540,-0.013062,-0.017995,18.0
ETHUSDT,15.074065,0.046413,0.020715,-0.007190,-0.017215,20.0
LINKUSDT,15.380846,0.045759,0.016698,-0.016935,-0.025541,23.0
LTCUSDT,16.809842,0.047733,0.015907,-0.016874,-0.026367,21.0
SOLUSDT,12.079430,0.048086,0.017302,-0.008020,-0.017296,21.0
TRXUSDT,119.576384,0.048816,-0.000174,-0.016803,-0.016635,18.5


## Get all Binance coin pairs

In [18]:
import requests
import pandas as pd

BINANCE_REST = "https://api.binance.com"

def get_spot_usdt_symbols():
    """All Spot symbols that trade against USDT and are currently TRADING."""
    info = requests.get(f"{BINANCE_REST}/api/v3/exchangeInfo", timeout=20).json()
    syms = []
    for s in info["symbols"]:
        if s.get("status") != "TRADING":
            continue
        if s.get("isSpotTradingAllowed") is not True:
            continue
        if s.get("quoteAsset") != "USDT":
            continue

        sym = s["symbol"]

        # Exclude leveraged tokens & some common non-spot-like tickers
        bad_substrings = ["UPUSDT", "DOWNUSDT", "BULLUSDT", "BEARUSDT", "3LUSDT", "3SUSDT", "5LUSDT", "5SUSDT"]
        if any(sym.endswith(x) for x in bad_substrings):
            continue

        syms.append(sym)
    return sorted(set(syms))

def rank_symbols_by_quote_volume(symbols):
    """Return DataFrame of symbols with 24h quoteVolume (USDT) sorted desc."""
    tickers = requests.get(f"{BINANCE_REST}/api/v3/ticker/24hr", timeout=20).json()
    # Build a map for fast lookup
    wanted = set(symbols)

    rows = []
    for t in tickers:
        sym = t.get("symbol")
        if sym not in wanted:
            continue
        # quoteVolume is in quoteAsset units, here USDT
        qv = float(t.get("quoteVolume", 0.0))
        rows.append({
            "symbol": sym,
            "quoteVolumeUSDT_24h": qv,
            "lastPrice": float(t.get("lastPrice", 0.0)),
            "priceChangePercent": float(t.get("priceChangePercent", 0.0)),
            "count": int(t.get("count", 0)),  # trade count 24h
        })

    df = pd.DataFrame(rows)
    df = df.sort_values("quoteVolumeUSDT_24h", ascending=False).reset_index(drop=True)
    return df

def get_top_usdt_pairs(n=100, min_quote_vol_usdt=None):
    """Top-N by 24h quote volume; optionally filter by minimum quote volume."""
    syms = get_spot_usdt_symbols()
    ranked = rank_symbols_by_quote_volume(syms)

    if min_quote_vol_usdt is not None:
        ranked = ranked[ranked["quoteVolumeUSDT_24h"] >= float(min_quote_vol_usdt)].copy()

    top = ranked.head(n).copy()
    return top, ranked


In [19]:
top100, ranked_all = get_top_usdt_pairs(n=100)
pairs = top100["symbol"].tolist()

top100.head(10), len(pairs)


(      symbol  quoteVolumeUSDT_24h    lastPrice  priceChangePercent    count
 0    BTCUSDT         5.497730e+08  88075.35000              -1.419  1579511
 1   USDCUSDT         4.915917e+08      1.00120               0.000   316253
 2    ETHUSDT         3.546146e+08   2910.05000              -1.704  1761157
 3   USD1USDT         3.291640e+08      1.00110              -0.020    37664
 4    SOLUSDT         1.693186e+08    123.11000              -3.048   474464
 5   FOGOUSDT         1.644840e+08      0.03705              -2.551  2219560
 6   ENSOUSDT         9.022044e+07      1.51000              10.461   606629
 7    XRPUSDT         7.267291e+07      1.86020              -2.867   338249
 8  FDUSDUSDT         6.737722e+07      0.99870               0.000    66478
 9   PAXGUSDT         6.116952e+07   5068.90000               0.255   104123,
 100)

In [ ]:
interval = "5m"
weeks = 52

paths = {}
dfs = {}

for p in pairs:
    db_path, df = load_or_fetch_pair_df(p, interval, weeks)
    paths[p] = db_path
    dfs[p] = df

len(dfs), list(dfs.keys())[:10]


[BTCUSDT] DB: BTCUSDT_5m_52weeks.db
[BTCUSDT] Loaded 104,832 rows from DB.
[USDCUSDT] DB: USDCUSDT_5m_52weeks.db
[USDCUSDT] No usable DB data found -> fetching from Binance...
